In [2]:
from gradio import Image, Label, Interface
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.applications import DenseNet121

# Define input image shape
img_shape = (224, 224, 3)

# Define your class labels here
classes = ['normal', 'smoking', 'spitting']

# Load pre-trained DenseNet121 model without the top layer
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=img_shape, pooling='max')

# Build your custom classification layers
model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(256, kernel_regularizer=tf.keras.regularizers.l2(l=0.016), 
          activity_regularizer=tf.keras.regularizers.l1(0.006),
          bias_regularizer=tf.keras.regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.45, seed=123),
    Dense(3, activation='softmax')
])

model.compile(Adamax(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# Load pre-trained weights into the model
model.load_weights("densenet121-Anomaly-behaviour-denseNet121-weights.h5")

# Define prediction function
def predict_image(image_file):
    # Resize the input image to match model input shape
    img_resized = tf.image.resize(image_file, (img_shape[0], img_shape[1]))
    # Make prediction
    predictions = model.predict(np.expand_dims(img_resized.numpy(), axis=0))
    # Get the predicted class
    predicted_class_index = np.argmax(predictions)
    predicted_class = classes[predicted_class_index]
    return {classes[i]: float(predictions[0][i]) for i in range(len(classes))}

# Interface
image_input = Image()  # Allows uploading PIL format images
label_output = Label()  # Displays top 3 predictions

# Launch the interface
Interface(fn=predict_image, inputs=image_input, outputs=label_output).launch(debug=True)


KeyboardInterrupt: 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 2s 2s/step


Traceback (most recent call last):
  File "/home/sankalp/.local/lib/python3.10/site-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/home/sankalp/.local/lib/python3.10/site-packages/gradio/route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/sankalp/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1847, in process_api
    result = await self.call_function(
  File "/home/sankalp/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1433, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/sankalp/.local/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/sankalp/.local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/home/sankalp/.local/lib/python3

Keyboard interruption in main thread... closing server.
